In [1]:
!python make_args.py --model_size bert-large-uncased

2023-05-26 09:41:34.977480: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 09:41:35.370618: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-26 09:41:35.370649: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-26 09:41:36.628348: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
import pickle


# open a file, where you ant to store the data
file = open('data/args.pkl', 'rb')

# dump information to that file
args=pickle.load(file)

# close the file
file.close()

On charge le modèle pré-entraîné:

In [1]:
from src.tasks.infer import infer_from_trained

inferer = infer_from_trained(args=None,detect_entities=True)

2023-06-09 10:44:32.078241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 10:44:32.506200: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-09 10:44:32.506248: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-09 10:44:33.694530: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Model config:  {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 0,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}



06/09/2023 10:44:47 AM [INFO]: Weights of BertModel not initialized from pretrained model: ['bert.classification_layer.weight', 'bert.classification_layer.bias']
06/09/2023 10:44:52 AM [INFO]: Loaded checkpoint model.
06/09/2023 10:44:53 AM [INFO]: Loaded model and optimizer.
06/09/2023 10:44:53 AM [INFO]: Done!


On recherche la relation entre les deux items marqués:

In [2]:
test = "The surprise [E1]visit[/E1] caused a [E2]frenzy[/E2] on the already chaotic trading floor."
inferer.infer_one_sentence(test)

Sentence:  The surprise [E1]visit[/E1] caused a [E2]frenzy[/E2] on the already chaotic trading floor.
Predicted:  Cause-Effect(e1,e2) 



11

On lui demande de détecter lui-même les items:

In [3]:
test="The surprise visit caused a frenzy on the already chaotic trading floor."
inferer.infer_sentence(test, detect_entities=True)

Sentence:  [E1]The surprise visit[/E1] caused [E2]a frenzy on the already chaotic trading floor[/E2] .
Predicted:  Cause-Effect(e1,e2) 

Sentence:  [E2]The surprise visit[/E2] caused [E1]a frenzy on the already chaotic trading floor[/E1] .
Predicted:  Cause-Effect(e2,e1) 



[11, 4]

Problème de non détection des items:

In [4]:
test2="Take the cube and put it in the black box."
inferer.infer_sentence(test2, detect_entities=True)

Found less than 2 entities!


Le problème vient de la bibliothèque Spacy que ne détecte pas bien certaines entités:

Par contre la méthode suivante semble plus simple et semble donner tous les groupes nominaux:

Il convient donc maintenant de fabriquer unes fonction pour fabriquer la liste des phrases annotées `[E1]...[/E1]` et `[E2]...[/E2]`

In [9]:
# Bibliothèques nécessaires
from itertools import combinations,permutations
import spacy

nlp = spacy.load("en_core_web_lg")

In [6]:
def add_markers(sentence, noun_chunk1, noun_chunk2):
    # Analyser la phrase en utilisant Spacy
    doc = nlp(sentence)
    
    # Trouver le début et la fin du noun_chunk dans la phrase
    start1 = noun_chunk1.start_char
    end1 = noun_chunk1.end_char
    start2 = noun_chunk2.start_char
    end2 = noun_chunk2.end_char
    
    if start1 < start2:
        # Ajouter les marqueurs "[C]" et "[/C]"
        modified_sentence = sentence[:start1] + "[E1]" + sentence[start1:end1] + "[/E1]" + sentence[end1:start2] + "[E2]" + sentence[start2:end2] + "[/E2]" + sentence[end2:]
    else:
        modified_sentence = sentence[:start2] + "[E2]" + sentence[start2:end2] + "[/E2]" + sentence[end2:start1] + "[E1]" + sentence[start1:end1] + "[/E1]" + sentence[end1:]

    
    return modified_sentence

# Exemple d'utilisation
sentence = "the cube is the box."
doc = nlp(sentence)
nc = list(doc.noun_chunks)
modified_sentence = add_markers(sentence, nc[1], nc[0])
print(modified_sentence)


[E2]the cube[/E2] is [E1]the box[/E1].


In [10]:
def get_annotated(sent):
    doc=nlp(sent)
    nps=doc.noun_chunks
    sentences=[]
    for np1, np2 in permutations([np for np in nps], 2):
        sentences.append(add_markers(sent,np1,np2))
    return sentences

get_annotated("In the beginning God created the heaven and the earth.")

['In [E1]the beginning[/E1] [E2]God[/E2] created the heaven and the earth.',
 'In [E1]the beginning[/E1] God created [E2]the heaven[/E2] and the earth.',
 'In [E1]the beginning[/E1] God created the heaven and [E2]the earth[/E2].',
 'In [E2]the beginning[/E2] [E1]God[/E1] created the heaven and the earth.',
 'In the beginning [E1]God[/E1] created [E2]the heaven[/E2] and the earth.',
 'In the beginning [E1]God[/E1] created the heaven and [E2]the earth[/E2].',
 'In [E2]the beginning[/E2] God created [E1]the heaven[/E1] and the earth.',
 'In the beginning [E2]God[/E2] created [E1]the heaven[/E1] and the earth.',
 'In the beginning God created [E1]the heaven[/E1] and [E2]the earth[/E2].',
 'In [E2]the beginning[/E2] God created the heaven and [E1]the earth[/E1].',
 'In the beginning [E2]God[/E2] created the heaven and [E1]the earth[/E1].',
 'In the beginning God created [E2]the heaven[/E2] and [E1]the earth[/E1].']

La méthode n'est pas parfaite car le modèle de recherche des relations se trompe, comme on le voit sur le lien entre les cubes et les sphères ci-dessous:

In [11]:
test5="You must put the cubes and the spheres in the boxe"
sentences=get_annotated(test5)
for sentence in sentences:
    print(inferer.infer_one_sentence(sentence))

Sentence:  [E1]You[/E1] must put [E2]the cubes[/E2] and the spheres in the boxe
Predicted:  Instrument-Agency(e2,e1) 

2
Sentence:  [E1]You[/E1] must put the cubes and [E2]the spheres[/E2] in the boxe
Predicted:  Instrument-Agency(e2,e1) 

2
Sentence:  [E1]You[/E1] must put the cubes and the spheres in [E2]the boxe[/E2]
Predicted:  Instrument-Agency(e2,e1) 

2
Sentence:  [E2]You[/E2] must put [E1]the cubes[/E1] and the spheres in the boxe
Predicted:  Other 

1
Sentence:  You must put [E1]the cubes[/E1] and [E2]the spheres[/E2] in the boxe
Predicted:  Entity-Destination(e1,e2) 

5
Sentence:  You must put [E1]the cubes[/E1] and the spheres in [E2]the boxe[/E2]
Predicted:  Entity-Destination(e1,e2) 

5
Sentence:  [E2]You[/E2] must put the cubes and [E1]the spheres[/E1] in the boxe
Predicted:  Other 

1
Sentence:  You must put [E2]the cubes[/E2] and [E1]the spheres[/E1] in the boxe
Predicted:  Entity-Destination(e1,e2) 

5
Sentence:  You must put the cubes and [E1]the spheres[/E1] in [E2]t

Idem ci-dessous ci-dessous:

In [12]:
test5="Each boxes must be filled with one cube and one sphere."
sentences=get_annotated(test5)
for sentence in sentences:
    print(inferer.infer_one_sentence(sentence))

Sentence:  [E1]Each boxes[/E1] must be filled with [E2]one cube[/E2] and one sphere.
Predicted:  Content-Container(e2,e1) 

16
Sentence:  [E1]Each boxes[/E1] must be filled with one cube and [E2]one sphere[/E2].
Predicted:  Content-Container(e2,e1) 

16
Sentence:  [E2]Each boxes[/E2] must be filled with [E1]one cube[/E1] and one sphere.
Predicted:  Content-Container(e1,e2) 

6
Sentence:  Each boxes must be filled with [E1]one cube[/E1] and [E2]one sphere[/E2].
Predicted:  Content-Container(e1,e2) 

6
Sentence:  [E2]Each boxes[/E2] must be filled with one cube and [E1]one sphere[/E1].
Predicted:  Content-Container(e1,e2) 

6
Sentence:  Each boxes must be filled with [E2]one cube[/E2] and [E1]one sphere[/E1].
Predicted:  Content-Container(e2,e1) 

16


Le modèle doit être améliorée:
- utiliser la méthode explicitée dans l'article en lien avec le Github pour un pré-entraînement non supervisé.

Ce modèle est cependant bon pour trouver distinguer conteneur et contenu:

In [13]:
test='[E1]the box[/E1] must be filled with [E2]the cubes[/E2]'
inferer.infer_one_sentence(test)

Sentence:  [E1]the box[/E1] must be filled with [E2]the cubes[/E2]
Predicted:  Content-Container(e2,e1) 



16

In [14]:
test='You must put [E1]the cubes[/E1] in [E2]the black box[/E2].'
inferer.infer_one_sentence(test)

Sentence:  You must put [E1]the cubes[/E1] in [E2]the black box[/E2].
Predicted:  Entity-Destination(e1,e2) 



5

In [15]:
test='You must put [E2]the cubes[/E2] in [E1]the black box[/E1].'
inferer.infer_one_sentence(test)

Sentence:  You must put [E2]the cubes[/E2] in [E1]the black box[/E1].
Predicted:  Other 



1

In [16]:
def add_markers(sentence, noun_chunk1, noun_chunk2):
    # Analyser la phrase en utilisant Spacy
    doc = nlp(sentence)
    
    # Trouver le début et la fin du noun_chunk dans la phrase
    start1 = sentence.find(noun_chunk1)
    end1 = start1+len(noun_chunk1)
    start2 = sentence.find(noun_chunk2)
    end2 = start2+len(noun_chunk2)
    
    if start1 < start2:
        # Ajouter les marqueurs "[C]" et "[/C]"
        modified_sentence = sentence[:start1] + "[E1]" + sentence[start1:end1] + "[/E1]" + sentence[end1:start2] + "[E2]" + sentence[start2:end2] + "[/E2]" + sentence[end2:]
    else:
        modified_sentence = sentence[:start2] + "[E2]" + sentence[start2:end2] + "[/E2]" + sentence[end2:start1] + "[E1]" + sentence[start1:end1] + "[/E1]" + sentence[end1:]

    
    return modified_sentence

test='You must put the cubes in the black box.'
add_markers(test,"the cubes","the black box")

'You must put [E1]the cubes[/E1] in [E2]the black box[/E2].'

In [17]:
def findContainer(sent,item1,item2):
    sent_marked=add_markers(sent,item1,item2)
    rel=inferer.infer_one_sentence(sent_marked)
    if rel==6 or rel==5:
        return 2
    elif rel==18 or rel==16:
        return 1
    else:
        return 0

In [18]:
findContainer("You must put the cubes in the black box","the cubes","the black box")

Sentence:  You must put [E1]the cubes[/E1] in [E2]the black box[/E2]
Predicted:  Entity-Destination(e1,e2) 



2